# **RPNet Tutorial**
The various dependencies required to run the **RPNet** can be easily installed using **pip**.<br>
It is recommended to run the program in a separate virtual environment using Anaconda with python version 3.9.<br>

**Note**: If you want to use a GPU, you must install **CUDA** libaray. The RPNet was developed using CUDA version 11.1.74.

**In terminal:**<br>
conda create -n rpnet python=3.9<br>
conda activate rpnet<br>
pip install rpnet<br>

RPNet supports multiprocessing-based preprocessing using the **parmap** module.<br>


Before using RPNet, download the pre-trained model from the link below and place it in the "../model" directory.<br>
[click here for pre-trained models](https://drive.google.com/drive/folders/1VlhPiLEx6XKBkmLdkc9RJ6fFTcSD0-0B?usp=sharing)

---
## 2. Example1 (polarity prediction)

Example 1 presents two earthquake events with magnitudes of 4.0 and 4.4, selected from the Kumamoto earthquake sequence as used in the original paper (Han et al., 2025).<br>
These events represent strike-slip and normal faulting mechanisms, respectively.<br><br>
Before running the RPNet, <span style="color:red"> you need to configure various variables and options in the `hyperparams.py` file.</span><br>
For detailed explanations of each variable, please refer to the comments within the file.

### 2.1 load modules

In [12]:
"""
# RPNet (v.0.1.0)
https://github.com/jongwon-han/RPNet

RPNet: Robust P-wave first-motion polarity determination using deep learning (Han et al., 2025; SRL)
doi: https://doi.org/10.1785/0220240384

Example script to run the sample Hi-net dataset

- Jongwon Han (@KIGAM)
- jwhan@kigam.re.kr
- Last update: 2025. 5. 15.
"""

#########################################################################################################

import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
from keras_self_attention import SeqSelfAttention
import matplotlib.pyplot as plt
import tqdm
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import subprocess
import shutil
from obspy import Stream, Trace
from obspy import UTCDateTime
from sklearn.model_selection import train_test_split
import plotly.figure_factory as ff
import matplotlib
import fnmatch
import time
from rpnet import *
from hyperparams import *
import glob

#########################################################################################################

---
### 2.2 Preparation for the prediction

In [13]:
# set gpu number
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_num

stime=time.time()

# make output directory / if exist remove it
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

# load raw data (catalog, phase, station files)
cat_df=pd.read_csv(event_catalog)
pha_df=pd.read_csv(phase_metadata)
sta_df=pd.read_csv(sta_metadata).sort_values(['sta']).reset_index(drop=True)
sta_df['sta0']=sta_df['sta']
pha_df['source']='original'

#### Earthquake catalog

In [14]:
cat_df

,event_id,data_id,jst,lat,lon,dep,mag,tmag,mag2,tmag2,strike,dip,slip,dist
0,J2016042016013028,D20160420001172,2016-04-20T16:01:30.280000Z,32.837667,130.799333,15.23,4.0,D,4.1,W,267,35,-91,9.34
1,J2016041507465204,D20160415000600,2016-04-15T07:46:52.040000Z,32.730000,130.797000,10.52,4.4,D,4.4,V,20,69,172,8.54


#### Phase information

In [15]:
pha_df

,event_id,data_id,sta,ptime,stime,pol,source
0,J2016041507465204,D20160415000600,N.TYNH,2016-04-15T07:46:54.740000Z,2016-04-15T07:46:56.650000Z,U,original
1,J2016041507465204,D20160415000600,KU.KMP,2016-04-15T07:46:55.820000Z,2016-04-15T07:46:58.470000Z,U,original
2,J2016041507465204,D20160415000600,KUIZU3,2016-04-15T07:46:56.080000Z,2016-04-15T07:46:59.080000Z,U,original
3,J2016041507465204,D20160415000600,N.YABH,2016-04-15T07:46:56.410000Z,2016-04-15T07:46:59.620000Z,U,original
4,J2016041507465204,D20160415000600,N.MSMH,2016-04-15T07:46:56.590000Z,2016-04-15T07:47:00.350000Z,D,original
...,...,...,...,...,...,...,...
116,J2016042016013028,D20160420001172,N.MYJH,2016-04-20T16:01:47.410000Z,2016-04-20T16:01:59.920000Z,U,original
117,J2016042016013028,D20160420001172,G.SIBI,2016-04-20T16:01:48.060000Z,2016-04-20T16:02:01.300000Z,U,original
118,J2016042016013028,D20160420001172,TAKAZA,2016-04-20T16:01:48.690000Z,2016-04-20T16:02:01.250000Z,U,original
119,J2016042016013028,D20160420001172,IKI,2016-04-20T16:01:53.980000Z,NaN,K,original


#### Station locations

In [16]:
sta_df

,net,sta,chan,lat,lon,elv,sta0
0,NaN,AGUNI,U,26.5927,127.2403,12.0,AGUNI
1,NaN,AIDA,U,34.9435,134.1653,170.0,AIDA
2,NaN,AIOI,U,33.7957,134.4488,165.0,AIOI
3,NaN,AKAIKE,U,33.7153,130.7928,130.0,AKAIKE
4,NaN,AKKESH,U,42.9987,144.6925,20.0,AKKESH
...,...,...,...,...,...,...,...
257,NaN,YONAGK,U,24.4511,122.9452,15.0,YONAGK
258,NaN,YONAGU,U,24.4672,123.0113,32.0,YONAGU
259,NaN,YORONJ,U,27.0246,128.4504,26.0,YORONJ
260,NaN,YTOYOT,U,34.2658,131.0622,120.0,YTOYOT


---
Use "add_sta" option when you want to utilize data from stations without picking information by estimating the P arrival time using TauP.

In [17]:
# add empty pick stations

if add_sta:
    z_files=sorted(glob.glob(wf_dir+'/*/*'))
    def get_add(z):
        id=z.split('/')[-2]
        sta=z.split('/')[-1].split('.U.')[0]
        if len(pha_df[(pha_df[fwfid]==id)&(pha_df['sta']==sta)])!=0:
            return pd.DataFrame()
        if not id in cat_df[fwfid].to_list():
            return pd.DataFrame()
        return pd.DataFrame({'pick':[id],'sta':[sta],fptime:[np.nan],fstime:[np.nan]})
    print('# get list of additional stations')
    results=parmap.map(get_add,z_files, pm_pbar=True, pm_processes=cores,pm_chunksize=1)
    pha_df0=pd.concat(results)
    pha_df0['source']='add'
    pha_df=pd.concat([pha_df,pha_df0]).reset_index(drop=True)

# get list of additional stations


100%|██████████| 100/100 [00:00<00:00, 4912.34it/s]


In [18]:
# Add station metadata to phase df
print('\n# Arrange metadata')
sta_df=sta_df[sta_df['sta'].isin(pha_df['sta'].to_list())].reset_index(drop=True)# Add station metadata to phase df
pha_df=pha_df[pha_df['sta'].isin(sta_df['sta'].to_list())].reset_index(drop=True)
pha_df['lat']=[sta_df[sta_df.sta==i]['lat'].iloc[0] for i in pha_df['sta'].to_list()]
pha_df['lon']=[sta_df[sta_df.sta==i]['lon'].iloc[0] for i in pha_df['sta'].to_list()]
pha_df['elv']=[sta_df[sta_df.sta==i]['elv'].iloc[0] for i in pha_df['sta'].to_list()]
# pha_df['net']=[sta_df[sta_df.sta==i]['net'].iloc[0] for i in pha_df['sta'].to_list()]
# pha_df['chan']=[sta_df[sta_df.sta==i]['chan'].iloc[0] for i in pha_df['sta'].to_list()]
sta_df['net']='HI' # Renaming, just for consistency

# make UTCDateTime objects
cat_df[ftime]=[UTCDateTime(i) for i in cat_df[ftime].to_list()]
print('- Done')


# Arrange metadata
- Done


---
Use the 'change2taup' option when you want to estimate the P arrival time using TauP.

In [19]:
# Change to TauP P arrival times (OPTION; considering pick uncertainty)
print(keep_initial_phase)
if change2taup:
    print('\n\n# change to TauP arrival')
    pha_df['ptime0']=pha_df[fptime]
    results=parmap.map(est_taup,[[idx,val,cat_df[cat_df[fwfid]==val[fwfid]].iloc[0],ftime,'P',taup_model,keep_initial_phase] for idx,val in pha_df.iterrows()]
                    , pm_pbar=True, pm_processes=cores,pm_chunksize=1)
    pha_df[fptime]=results
    print('- TauP (P) Done')

    print('# change to TauP S arrival')
    pha_df['stime0']=pha_df[fstime]
    results=parmap.map(est_taup,[[idx,val,cat_df[cat_df[fwfid]==val[fwfid]].iloc[0],ftime,'S',taup_model,keep_initial_phase] for idx,val in pha_df.iterrows()]
                    , pm_pbar=True, pm_processes=cores,pm_chunksize=1)
    pha_df[fstime]=results
    print('- TauP (S) Done')

pha_df=pha_df.sort_values([fwfid,'sta']).reset_index(drop=True)
pha_df

False


# change to TauP arrival


100%|██████████| 100/100 [00:00<00:00, 175.78it/s]

- TauP (P) Done
# change to TauP S arrival



100%|██████████| 100/100 [00:00<00:00, 173.33it/s]

- TauP (S) Done


,event_id,data_id,sta,ptime,stime,pol,source,lat,lon,elv,ptime0,stime0
0,J2016041507465204,D20160415000600,AKAIKE,2016-04-15T07:47:11.000967Z,2016-04-15T07:47:24.770273Z,K,original,33.7153,130.7928,130.0,2016-04-15T07:47:10.540000Z,NaN
1,J2016041507465204,D20160415000600,BEPPUA,2016-04-15T07:47:07.343610Z,2016-04-15T07:47:18.456997Z,K,original,33.3363,131.4072,434.0,2016-04-15T07:47:07.140000Z,NaN
2,J2016041507465204,D20160415000600,HICHIY,2016-04-15T07:47:06.778217Z,2016-04-15T07:47:17.481087Z,K,original,32.4431,131.6368,53.0,2016-04-15T07:47:06.640000Z,NaN
3,J2016041507465204,D20160415000600,HIKIMI,2016-04-15T07:47:26.284081Z,2016-04-15T07:47:52.559933Z,K,original,34.5345,131.9267,350.0,2016-04-15T07:47:26.940000Z,NaN
4,J2016041507465204,D20160415000600,HIROMI,2016-04-15T07:47:20.393597Z,2016-04-15T07:47:41.965419Z,K,original,33.2170,132.6220,450.0,2016-04-15T07:47:20.740000Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,J2016042016013028,D20160420001172,TAKAZA,2016-04-20T16:01:48.653920Z,2016-04-20T16:02:02.060362Z,U,original,31.9047,131.0857,170.0,2016-04-20T16:01:48.690000Z,2016-04-20T16:02:01.250000Z
96,J2016042016013028,D20160420001172,TAMANA,2016-04-20T16:01:35.911039Z,2016-04-20T16:01:40.000247Z,U,original,32.9670,130.5305,230.0,2016-04-20T16:01:35.880000Z,2016-04-20T16:01:39.910000Z
97,J2016042016013028,D20160420001172,TSUNO,2016-04-20T16:01:46.461043Z,2016-04-20T16:01:58.256864Z,K,original,32.2490,131.5022,130.0,2016-04-20T16:01:45.970000Z,NaN
98,J2016042016013028,D20160420001172,URESHI,2016-04-20T16:01:45.081953Z,2016-04-20T16:01:55.830333Z,K,original,33.0965,129.9467,160.0,2016-04-20T16:01:44.670000Z,NaN


---
The input data (numpy matrix) is generated in 4-second segments from continuous waveform data (SAC or MSEED).

In [20]:
# Make input data matrix
print('\n\n# Make input matrix from waveform data')
results=parmap.map(wf2matrix,[[idx,val,fwfid,fptime,wf_dir,out_dir] for idx, val in pha_df.iterrows()], pm_pbar=True, pm_processes=cores,pm_chunksize=1)
results=[i for i in results if i is not None]
a,b=zip(*results)
pha_df=pha_df.iloc[list(a)].reset_index(drop=True)
in_mat=np.vstack(b)
print('% calculation time (min): ','%.2f'%((time.time()-stime)/60))
print('- Done')



# Make input matrix from waveform data


100%|██████████| 100/100 [00:00<00:00, 1014.87it/s]

% calculation time (min):  0.03
- Done


---
### 2.3 Polarity prediction (main)

In [21]:
# RPNet main prediction
print('\n\n# Predict polarity (RPNet)')
r_df=pred_rpnet(model,in_mat,pha_df,batch_size=batch_size,iteration=iteration,gpu_num=gpu_num,time_shift=0.0,mid_point=250)
print('% calculation time (min): ','%.2f'%((time.time()-stime)/60))
r_df.to_csv(out_dir+'/pol_result.csv',index=False)
print('- Done')



# Predict polarity (RPNet)
# iterate prediction


100%|██████████| 100/100 [00:07<00:00, 13.00it/s]

% calculation time (min):  0.16
- Done


---
### 2.4 From RPNet's polarity result to SKHASH input format

In [22]:
# Renaming station code (only for Hi-net)
sta_df['net']='HI'
sta_df['chan']=sta_df['chan'].replace('U','HHZ')
sta_df['sta']=['S'+str(i+1).rjust(3,'0') for i in range(len(sta_df))]
cat_df=cat_df
pha_df=pha_df[pha_df['data_id'].isin(cat_df['data_id'].to_list())].reset_index(drop=True)
r_df=pd.read_csv(out_dir+'/pol_result.csv')
r_df=r_df[r_df['data_id'].isin(cat_df['data_id'].to_list())].reset_index(drop=True)


# Make SKHASH input setting
if iteration!=0:
    r_df.loc[r_df['std'] > std_threshold, 'predict'] = 'K'
if rm_unknwon:
    r_df=r_df[r_df['predict']!='K'].reset_index(drop=True)
# make threshold for mean
if iteration!=0 and mean_threshold!=0:
    r_df=r_df[r_df['prob']>=mean_thresuld].reset_index(drop=True)
print('\n\n# Final result:')
print(r_df.to_string())

r_df=r_df.drop_duplicates(['sta',fwfid]).reset_index(drop=True)
prep_skhash(cat_df=cat_df,pol_df=r_df,amp=[],sta_df=sta_df,ftime=ftime,fwfid=fwfid,ctrl0=ctrl0,out_dir=out_dir,hash_version=hash_version)
print('% calculation time (min): ','%.2f'%((time.time()-stime)/60))
print('\n\n@ ALL DONE!')




# Final result:
             event_id          data_id     sta                        ptime                        stime pol    source      lat       lon    elv                       ptime0                       stime0 predict    prob     std
0   J2016041507465204  D20160415000600  AKAIKE  2016-04-15T07:47:11.000967Z  2016-04-15T07:47:24.770273Z   K  original  33.7153  130.7928  130.0  2016-04-15T07:47:10.540000Z                          NaN       U  0.9911  0.0224
1   J2016041507465204  D20160415000600  BEPPUA  2016-04-15T07:47:07.343610Z  2016-04-15T07:47:18.456997Z   K  original  33.3363  131.4072  434.0  2016-04-15T07:47:07.140000Z                          NaN       D  0.8159  0.1290
2   J2016041507465204  D20160415000600  HICHIY  2016-04-15T07:47:06.778217Z  2016-04-15T07:47:17.481087Z   K  original  32.4431  131.6368   53.0  2016-04-15T07:47:06.640000Z                          NaN       D  1.0000  0.0000
3   J2016041507465204  D20160415000600  HIKIMI  2016-04-15T07:47:26.284081

---
## 3. Focal Mechanism Calculation (SKHASH)

RPNet does not provide the source code for **SKHASH** directly.<br>
Please refer to the link below to download and install SKHASH.<br>

[click here for SKHASH](https://code.usgs.gov/esc/SKHASH)

However, the RPNet's conda environment is configured to support SKHASH, so you can run the SKHASH.py script within the same RPNet environment without the need for an additional virtual environment setup.

Before running the RPNet, properly configure the **control_file0.txt** for SKHASH under **"ctrl0"** in hyperparams.py. The default file will be copied to the RPNet result directory, where the necessary settings for running SKHASH will be automatically generated.

**Run in terminal:**<br>
python SKHASH.py ./output01/hash2/control_file.txt<br>

Please check "./output01/hash2/OUT" directory after running SKHASH for the focal mechanism result.

The MSEED files in the output directory contain the trimmed 4-second waveform segments that were actually used by RPNet for polarity prediction.
